# Mountain car #

# I add the modified reward function and applied to the training.
# But the training result stop increasing around -0.3

## Imports ##

In [ ]:
import gym
import tensorflow as tf
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

## Environment ##

In [2]:
# Create the Mountain Car game environment
env = gym.make('MountainCar-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Hyperparameters ##

In [3]:
# Environment parameters
state_size = 2      # state size 2
action_size = 3

hidden_layer_size = 128

batch_size = 25

learning_rate = 0.01

max_episodes = 100

max_steps = 200
percentile = 70

## Neural network ##

In [4]:
class Net:
    def __init__(self, 
                 state_size = state_size, 
                 action_size = action_size, 
                 hidden_layer_size = hidden_layer_size,
                 learning_rate = learning_rate, 
                 name = 'net'):
        
        with tf.variable_scope(name):
        
            ### Prediction part
        
            # Input layer, state s is input
            self.states = tf.placeholder(
                tf.float32, 
                [None, state_size])
            
            # Hidden layer, ReLU activation
            self.hidden_layer = tf.contrib.layers.fully_connected(
                self.states, 
                hidden_layer_size)
            
            # Hidden layer, linear activation, logits
            self.logits = tf.contrib.layers.fully_connected(
                self.hidden_layer, 
                action_size,
                activation_fn = None)
            
            # Output layer, softmax activation yields probability distribution for actions
            self.probabilities = tf.nn.softmax(self.logits)
    
            ### Training part 
    
            # Action a
            self.actions = tf.placeholder(
                tf.int32, 
                [None])
            
            # One-hot encoded action a 
            #
            # encoded_action_vector = [1, 0] if action a = 0
            # encoded_action_vector = [0, 1] if action a = 1
            self.one_hot_actions = tf.one_hot(
                self.actions, 
                action_size)

            # cross entropy
            self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
                logits = self.logits, 
                labels = self.one_hot_actions)
            
            # cost
            self.cost = tf.reduce_mean(self.cross_entropy)
            
            # Optimizer
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
            
    # get action chosen according to current probabilistic policy
    def get_action(self, state):
        feed_dict = { self.states : np.array([state]) } 
        probabilities = sess.run(self.probabilities, feed_dict = feed_dict)
        
        return np.random.choice(action_size, p=probabilities[0])
    
    # train based on batch
    def train(self, batch):
        states, actions = zip(*batch)
        states = np.array(states)
        actions = np.array(actions)
        
        feed_dict = {
            self.states : states,
            self.actions : actions
        }
        
        sess.run(self.optimizer, feed_dict = feed_dict)

## Training ##

In [5]:
tf.reset_default_graph()
net = Net(name = 'net',
          hidden_layer_size = hidden_layer_size,
          learning_rate = learning_rate)


In [6]:
def compute_modified_reward(next_state):
    modified_reward = np.square(max(0, next_state[0] + 0.5))
    if next_state[0] >= 0.5: 
        modified_reward += 1.0
    return modified_reward

In [7]:
import random
import bisect
import time

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    start_index = int(max_episodes * percentile / 100)
    
    while True:

        total_reward_list = []
        trajectory_list = []

        for e in np.arange(max_episodes):
            total_reward = 0.0
            trajectory = []
            state = env.reset()
            for s in np.arange(max_steps):
                action = net.get_action(state)
                next_state, _, done, _ = env.step(action)
                modified_reward = compute_modified_reward(next_state)
                total_reward += modified_reward
                trajectory.append((state, action))
                state = next_state
                if done: break

            index = bisect.bisect(total_reward_list, total_reward)
            total_reward_list.insert(index, total_reward)
            trajectory_list.insert(index, trajectory)
        
        # keep the elite episodes, that is, throw out the bad ones 
        # train on state action pairs extracted from the elite episodes
        # this code is not optimized, it can be cleaned up 
        state_action_pairs = []
        for trajectory in trajectory_list[start_index:]:
            for state_action_pair in trajectory:
                state_action_pairs.append(state_action_pair)
        # shuffle to avoid correlations between adjacent states
        random.shuffle(state_action_pairs) 
        n = len(state_action_pairs)
        batches = [state_action_pairs[k:k + batch_size] for k in np.arange(0, n, batch_size)]

        for batch in batches:
            net.train(batch)

        # test agent
        state = env.reset()
        # env.render()
        # time.sleep(0.05)
        total_reward = 0.0
        for s in np.arange(max_steps):
            action = net.get_action(state)
            # state, reward, done, _ = env.step(action)
            # total_reward += reward
            state, _, done, _ = env.step(action)
            modified_reward = compute_modified_reward(state)
            total_reward += modified_reward
            # env.render()
            # time.sleep(0.05)
            if done: break

        env.close()
        print("End position:", state[0])
        
        if state[0] >= 0.5:
            break    

End position: -0.6448342391113898


End position: -0.4562627377788806


End position: -0.3754537453623141


End position: -0.6085727443430691


End position: -0.42130619360469296


...

End position: -0.3565490360535902


End position: -0.17709899358062947


End position: -0.2955915579324977


End position: -0.2837313832966929


KeyboardInterrupt: 